In [2]:
# import rocrate models
import os
import sys

#sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__file__), '../')))
#os.path.join(os.path.dirname(__file__), '../')
srcPath = os.path.abspath('C:\\Users\\Max\\Documents\\GitHub\\mds_python\\src\\' )
sys.path.insert(0, srcPath)


#import fairscape_mds.models.rocrate as rocrate


In [3]:
import pathlib
import json

In [46]:
crateAPMSLoader = pathlib.Path('data/1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha/')
crateMetadataPathNew = crateAPMSLoader / 'ro-crate-metadata.json'
crateMetadataPathOld = crateAPMSLoader / 'old-ro-crate-metadata.json'


In [4]:
from typing import (
	List,
	Dict,
	Callable
)

In [ ]:
formattedCrateMetadataGraph = crateMetadataNew['@graph'].copy()
rootElem = findRootElem(formattedCrateMetadataGraph)

In [107]:
def getIndexGraphFilter(queryFunc: callable, graph: List):
	indexList = [ i for i, elem in  enumerate(graph) if queryFunc(elem)]
	if len(indexList) != 1:
		raise Exception
	else:
		return indexList[0]


In [5]:
class ROCrateFilterException(Exception):
	def __init__(message, results):
		self.results = results
		self.message = message
		super().__init__(self.message)


def filterOneMetadataGraph(metadataGraph: List[Dict], filterFunc: Callable):
	metadataSearch = list(filter(filterFunc, metadataGraph))

	if len(metadataSearch) < 1:
		# raise error if more than one elem
		raise ROCrateFilterException('More than one element found', metadataSearch)
	elif len(metadataSearch) == 0:
		return None
	else:	
		metadataElem = metadataSearch[0]
		return metadataElem


def findRootElem(crateMetadata)->Dict:
	crateMetadataFilter = lambda x: x.get('@id') == 'ro-crate-metadata.json'
	crateGUIDFilter = lambda elem: elem.get('@id') == rootMetadataGUID

	# find the ro-crate-metadata.json elem
	roCrateMetadataElem = filterOneMetadataGraph(crateMetadataNew['@graph'], crateMetadataFilter)
	rootMetadataGUID = roCrateMetadataElem.get('about', {}).get('@id')

	rootMetadataElem = filterOneMetadataGraph(crateMetadataNew['@graph'], crateGUIDFilter)

	return rootMetadataElem

def getIndexGraphFilter(queryFunc: callable, graph: List):
	indexList = [ i for i, elem in  enumerate(graph) if queryFunc(elem)]
	if len(indexList) != 1:
		raise Exception
	else:
		return indexList[0]


def pruneROCrate(metadataGraph: List, filterFunc: callable):
	elemIndex = getIndexGraphFilter(filterFunc, metadataGraph)
	metadataGraph.pop(elemIndex)

def formatROCrateToModel(inputCrate: Dict)-> rocrate.ROCrate:
	""" Converts ROCrate v1.1 metadata into a fairscape ROCrate pydantic model

	:param inputCrate: input metadata
	:type inputCrate: dict
	:return: metadata formatted as an ROCrate model
	:rtype: fairscape_mds.models.rocrate.ROCrate
	""" 
	formattedCrateMetadataGraph = inputCrate['@graph'].copy()
	rootElem = findRootElem(formattedCrateMetadataGraph)
	rootMetadataGUID = rootElem.get("@id")

	# pop rocrate elem
	pruneROCrate(formattedCrateMetadataGraph, lambda x: x.get("@id") == rootMetadataGUID)
	pruneROCrate(formattedCrateMetadataGraph, lambda x: x.get("@id") == 'ro-crate-metadata.json')
	pruneROCrate(formattedCrateMetadataGraph, lambda x: x.get("@type") == 'Project')
	pruneROCrate(formattedCrateMetadataGraph, lambda x: x.get("@type") == 'Organization')

	# set the metadata graph
	rootMetadataElem['@graph'] = formattedCrateMetadataGraph

	# overwrite the metadata graph
	rootMetadataElem['@type'] = 'https://w3id.org/EVI#ROCrate'

	# validate that root metadata elem works for rocrate
	return rocrate.ROCrate.model_validate(rootMetadataElem)

NameError: name 'rocrate' is not defined

In [25]:
processROCrate(crateMetadataNew)

NameError: name 'processROCrate' is not defined

## Testing ROCrateV1_1 Models

In [6]:
# import rocrate models
import os
import sys

#sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__file__), '../')))
#os.path.join(os.path.dirname(__file__), '../')
srcPath = os.path.abspath('C:\\Users\\Max\\Documents\\GitHub\\mds_python\\src\\' )
sys.path.insert(0, srcPath)


In [7]:
import pathlib
import json

crateAPMSLoader = pathlib.Path('data/2.cm4ai_chromatin_mda-mb-468_untreated_imageembedfold1_initialrun0.1alpha')
crateMetadataPathNew = crateAPMSLoader / 'ro-crate-metadata.json'
crateMetadataPathOld = crateAPMSLoader / 'old-ro-crate-metadata.json'

with crateMetadataPathNew.open('r') as crateFile:
	crateMetadataNew = json.load(crateFile)

with crateMetadataPathOld.open('r') as crateFile:
	crateMetadataOld = json.load(crateFile)

In [8]:
from pydantic import BaseModel, Field
from typing import Optional, Union, Dict, List, Literal
from fairscape_mds.models.rocrate import ROCrateDataset, ROCrateSoftware, ROCrateComputation
from fairscape_mds.models.fairscape_base import IdentifierValue

class ROCrateOrganization(IdentifierValue):
    metadataType: Literal['Organization'] = Field(alias="@type")
    name: str

class ROCrateProject(IdentifierValue):
    metadataType: Literal['Project'] = Field(alias="@type")
    name: str

class ROCrateMetadataFileElem(BaseModel):
    """Metadata Element of an ROCrate cooresponding to the `ro-crate-metadata.json` file itself

    Example

        ```
        {
            "@id": "ro-crate-metadata.json",
            "@type": "CreativeWork",
            "conformsTo": {
                "@id": "https://w3id.org/ro/crate/1.2-DRAFT"
            },
            "about": {
                "@id": "https://fairscape.net/ark:59852/rocrate-2.cm4ai_chromatin_mda-mb-468_untreated_apmsembed_initialrun0.1alpha"
            }
        }
        ```
    """
    guid: str = Field(alias="@id")
    metadataType: Literal["CreativeWork"] = Field(alias="@type")
    conformsTo: IdentifierValue
    about: IdentifierValue

class ROCrateMetadataElem(BaseModel):
    """Metadata Element of ROCrate that represents the crate as a whole

    Example
        ```
        {
            '@id': 'https://fairscape.net/ark:59852/rocrate-2.cm4ai_chromatin_mda-mb-468_untreated_imageembedfold1_initialrun0.1alpha',
            '@type': ['Dataset', 'https://w3id.org/EVI#ROCrate'],
            'name': 'Initial integration run',
            'description': 'Ideker Lab CM4AI 0.1 alpha MDA-MB-468 untreated chromatin Initial integration run IF Image Embedding IF microscopy images embedding fold1',
            'keywords': ['Ideker Lab',
            'CM4AI',
            '0.1 alpha',
            'MDA-MB-468',
            'untreated',
            'chromatin',
            'Initial integration run',
            'IF Image Embedding',
            'IF microscopy images',
            'embedding',
            'fold1'],
            'isPartOf': [
                {'@id': 'ark:/Ideker_Lab'}, 
                {'@id': 'ark:/Ideker_Lab/CM4AI'}
                ],
            'version': '0.5alpha',
            'license': 'https://creativecommons.org/licenses/by-nc-sa/4.0/deed.en',
            'associatedPublication': 'Clark T, Schaffer L, Obernier K, Al Manir S, Churas CP, Dailamy A, Doctor Y, Forget A, Hansen JN, Hu M, Lenkiewicz J, Levinson MA, Marquez C, Mohan J, Nourreddine S, Niestroy J, Pratt D, Qian G, Thaker S, Belisle-Pipon J-C, Brandt C, Chen J, Ding Y, Fodeh S, Krogan N, Lundberg E, Mali P, Payne-Foster P, Ratcliffe S, Ravitsky V, Sali A, Schulz W, Ideker T. Cell Maps for Artificial Intelligence: AI-Ready Maps of Human Cell Architecture from Disease-Relevant Cell Lines. BioRXiv 2024.',
            'author': ['Clark T',
            'Schaffer L',
            'Obernier K',
            'Al Manir S',
            'Churas CP',
            'Dailamy A',
            'Doctor Y',
            'Forget A',
            'Hansen JN',
            'Hu M',
            'Lenkiewicz J',
            'Levinson MA',
            'Marquez C',
            'Mohan J',
            'Nourreddine S',
            'Niestroy J',
            'Pratt D',
            'Qian G',
            'Thaker S',
            'Belisle-Pipon J-C',
            'Brandt C',
            'Chen J',
            'Ding Y',
            'Fodeh S',
            'Krogan N',
            'Lundberg E',
            'Mali P',
            'Payne-Foster P',
            'Ratcliffe S',
            'Ravitsky V',
            'Sali A',
            'Schulz W',
            'Ideker T.'],
            'conditionsOfAccess': 'This dataset was created by investigators and staff of the Cell Maps for Artificial Intelligence project (CM4AI - https://cm4ai.org), a Data Generation Project of the NIH Bridge2AI program, and is copyright (c) 2024 by The Regents of the University of California and, for cellular imaging data, by The Board of Trustees of the Leland Stanford Junior University. It is licensed for reuse under the Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International (CC-BY-NC-SA 4.0) license, whose terms are summarized here: https://creativecommons.org/licenses/by-nc-sa/4.0/deed.en.  Proper attribution credit as required by the license includes citation of the copyright holders and of the attribution parties, which includes citation of the following article: Clark T, Schaffer L, Obernier K, Al Manir S, Churas CP, Dailamy A, Doctor Y, Forget A, Hansen JN, Hu M, Lenkiewicz J, Levinson MA, Marquez C, Mohan J, Nourreddine S, Niestroy J, Pratt D, Qian G, Thaker S, Belisle-Pipon J-C, Brandt C, Chen J, Ding Y, Fodeh S, Krogan N, Lundberg E, Mali P, Payne-Foster P, Ratcliffe S, Ravitsky V, Sali A, Schulz W, Ideker T. Cell Maps for Artificial Intelligence: AI-Ready Maps of Human Cell Architecture from Disease-Relevant Cell Lines. BioRXiv 2024.”',
            'copyrightNotice': 'Copyright (c) 2024 by The Regents of the University of California',
            'hasPart': [
                {'@id': 'https://fairscape.net/ark:59852/software-cellmaps_image_embedding-N2ux5jg'},
                {'@id': 'https://fairscape.net/ark:59852/dataset-cellmaps_image_embedding-output-file-N2ux5jg'},
                {'@id': 'https://fairscape.net/ark:59852/dataset-Densenet-model-file-N2ux5jg'},
                {'@id': 'https://fairscape.net/ark:59852/computation-IF-Image-Embedding-N2ux5jg'}
            ]
        }
        ```
    """ 
    guid: str = Field(alias="@id")
    metadataType: List[str] = Field(alias="@type")
    name: str
    keywords: List[str]
    isPartOf: List[IdentifierValue]
    version: str
    dataLicense: str = Field(alias="license")
    associatedPublication: str
    author: Union[str, List[str]]
    conditionsOfAccess: str
    copyrightNotice: str
    hasPart: List[IdentifierValue]
    
    pass

class ROCrateV1_2(BaseModel):
    context: Optional[Dict] = Field(alias="@context")
    metadataGraph: List[Union[
        ROCrateDataset,
        ROCrateSoftware,
        ROCrateComputation,
        ROCrateMetadataElem,
        ROCrateMetadataFileElem,
        ROCrateProject,
        ROCrateOrganization
    ]] = Field(alias="@graph")

c:\Users\Max\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [16]:

#cratePath = 'data/cm4ai_chromatin_mda-mb-468_paclitaxel_ifimage_0.1_alpha/ro-crate-metadata.json'
#cratePath = 	'data/cm4ai_chromatin_mda-mb-468_vorinostat_ifimage_0.1_alpha/ro-crate-metadata.json'
cratePath = 'data/cm4ai_chromatin_mda-mb-468_untreated_apms_0.1_alpha/ro-crate-metadata.json'

with open(cratePath, 'r') as crateFile:
	crateMetadata = json.load(crateFile)

In [17]:
#crateMetadata['@graph'][2884]
#crateMetadata['@graph'][3140]

parsedMetadata = ROCrateV1_2.model_validate(crateMetadata)

In [15]:
#debugging rocrates
# cratePath = 'data/1.cm4ai_chromatin_mda-mb-468_untreated_imageloader_initialrun0.1alpha/ro-crate-metadata.json'
#cratePath = 'data/2.cm4ai_chromatin_mda-mb-468_untreated_imageembedfold1_initialrun0.1alpha/ro-crate-metadata.json'
#cratePath = 'data/2.cm4ai_chromatin_mda-mb-468_untreated_imageembedfold2_initialrun0.1alpha/ro-crate-metadata.json'
#cratePath = 'data/3.cm4ai_chromatin_mda-mb-468_untreated_coembedfold1_initialrun0.1alpha/ro-crate-metadata.json'
#cratePath = 'data/4.cm4ai_chromatin_mda-mb-468_untreated_hierarchy_initialrun0.1alpha/ro-crate-metadata.json'
#cratePath = 'data/cm4ai_chromatin_mda-mb-468_paclitaxel_ifimage_0.1_alpha/ro-crate-metadata.json'
#cratePath = 	'data/cm4ai_chromatin_mda-mb-468_vorinostat_ifimage_0.1_alpha/ro-crate-metadata.json'
cratePath = 'data/cm4ai_chromatin_mda-mb-468_untreated_apms_0.1_alpha/ro-crate-metadata.json'

index = 4

with open(cratePath, 'r') as crateFile:
	crateMetadata = json.load(crateFile)

computationElement = crateMetadata['@graph'][index].copy()

newGenerated = [{"@id": elem} for elem in computationElement['generated']]
computationElement['generated'] = newGenerated

computationElement['usedSoftware'] = [{"@id": elem} for elem in computationElement['usedSoftware']]
computationElement['usedDataset'] = [{"@id": elem} for elem in computationElement['usedDataset']]

# overwrite the list
crateMetadata['@graph'][index] = computationElement

with open(cratePath, 'w') as crateFile:
	json.dump(crateMetadata, crateFile, indent=2)

In [10]:

with open(cratePath, 'r') as crateFile:
	crateMetadata = json.load(crateFile)


for elemIndex in [i for i, elem in enumerate(crateMetadata['@graph']) if elem.get("@type") == "Dataset"]:
	crateMetadata['@graph'][elemIndex]['generatedBy'] = [
		{"@id": crateMetadata['@graph'][elemIndex]['generatedBy']}
	] 


with open(cratePath, 'w') as crateFile:
	json.dump(crateMetadata, crateFile, indent=2)

In [20]:
allCrates = [
	'data/1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha',
	'data/1.cm4ai_chromatin_mda-mb-468_untreated_imageloader_initialrun0.1alpha',
	'data/2.cm4ai_chromatin_mda-mb-468_untreated_apmsembed_initialrun0.1alpha',
	'data/2.cm4ai_chromatin_mda-mb-468_untreated_imageembedfold1_initialrun0.1alpha',
	'data/2.cm4ai_chromatin_mda-mb-468_untreated_imageembedfold2_initialrun0.1alpha',
	'data/3.cm4ai_chromatin_mda-mb-468_untreated_coembedfold1_initialrun0.1alpha',
	'data/3.cm4ai_chromatin_mda-mb-468_untreated_coembedfold2_initialrun0.1alpha',
	'data/4.cm4ai_chromatin_mda-mb-468_untreated_hierarchy_initialrun0.1alpha',
	'data/cm4ai_chromatin_mda-mb-468_paclitaxel_ifimage_0.1_alpha',
	'data/cm4ai_chromatin_mda-mb-468_vorinostat_ifimage_0.1_alpha',
	'data/cm4ai_chromatin_mda-mb-468_untreated_apms_0.1_alpha',
]

for index, crate in enumerate(allCrates):
	print(index)
	cratePath = pathlib.Path(crate) / 'ro-crate-metadata.json'

	with cratePath.open('r') as crateFile:
		crateMetadataNew = json.load(crateFile)	
		parsedMetadata = ROCrateV1_2.model_validate(crateMetadataNew)
	print(f"Success: {str(cratePath)}")

0
Success: data\1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha\ro-crate-metadata.json
1
Success: data\1.cm4ai_chromatin_mda-mb-468_untreated_imageloader_initialrun0.1alpha\ro-crate-metadata.json
2
Success: data\2.cm4ai_chromatin_mda-mb-468_untreated_apmsembed_initialrun0.1alpha\ro-crate-metadata.json
3
Success: data\2.cm4ai_chromatin_mda-mb-468_untreated_imageembedfold1_initialrun0.1alpha\ro-crate-metadata.json
4
Success: data\2.cm4ai_chromatin_mda-mb-468_untreated_imageembedfold2_initialrun0.1alpha\ro-crate-metadata.json
5
Success: data\3.cm4ai_chromatin_mda-mb-468_untreated_coembedfold1_initialrun0.1alpha\ro-crate-metadata.json
6
Success: data\3.cm4ai_chromatin_mda-mb-468_untreated_coembedfold2_initialrun0.1alpha\ro-crate-metadata.json
7
Success: data\4.cm4ai_chromatin_mda-mb-468_untreated_hierarchy_initialrun0.1alpha\ro-crate-metadata.json
8
Success: data\cm4ai_chromatin_mda-mb-468_paclitaxel_ifimage_0.1_alpha\ro-crate-metadata.json
9
Success: data\cm4ai_chroma

In [ ]:
# convert metadata into 1.1 format

def formatModelToROCrate(inputMetadata: Dict)-> rocrate.ROCrateV1_1:
	pass

In [ ]:
class ConvertROCrate():

	def __init__(inputMetadata: dict):
		self.inputMetadata = inputMetadata
		pass

	def toReadModel(self):
		pass

	def toROCrate(self):
		pass